<a id='TOC'></a>

# Table of Contents
1. [Things to Do](#Things-to-Do)
1. [Introduction](#Introduction)
1. [Set up Python](#Set-up-Python)
2. [Functions](#Functions)
3. [Getting Data](#Get-Data)
4. [Analyze Data](#Analyze-Data):
    - [Population Size](#Population-Size)
    - [Species Distribution](#Species-Distribution)
    - [Sex Distribution](#Sex-Distribution)
    - [Tail Condition Distribution](#Tail-Condition-Distribution)
    - [Location](#Location)
    - [Morphometrics](#Morphometrics):
        - [SVL](#SVL)
        - [TL](#TL)
        - [RTL](#RTL)
        - [Mass](#Mass)
    - [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
    - [Captures](#Captures)
    - [Growth](#Growth)
        - [SVL Growth](#SVL-Growth)
5. [Export Files](#Export-Files)

# Things to Do
- Filter and use only the first capture in a year for each animal for metrics

## [Resume Here](#resume)

## Introduction

This notebook contains code and output of descriptive analyses for the 2000-2017 CC dataset after cleaning.

The objectives of this notebook are to describe the community of the _Sceloporus jarrovii_ and _Sceloporus virgatus_ lizards in the Crystal Creek wash from 2000 until 2017.  The population demographic metrics we examine are: [population size](#Population-Size), [sex distribution](#Sex-Distribution), [tail condition distribution](#Tail-Condition-Distribution), [location](#Location), [morphometrics](#Morphometrics) -- [SVL](#SVL), [TL](#TL), [RTL](#RTL), [mass](#Mass) --,  [survival and rates and likelihood of recapture](#Survival-and-Rates-and-Likelihood-of-Recapture), and [growth](#Growth).

We will examine these metrics and interactions among them with particular interest in the impact of environmental factors from year to year.


##  Set up Python

First we will need to set up the python environment, importing the necessary packages and setting the display options.

[Top](#TOC)

In [1]:
import pandas as pd
import numpy as np
import os, glob, logging
from summary_functions import *
from scipy import stats
from monthlit import *


import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
plotly.tools.set_config_file(world_readable=True)


# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 50

### Setting File Locations

In [2]:
deviceDict = {'dataBig':{'source':'S:/Chris/TailDemography/TailDemography/outputFiles'
                         ,'log':'S:/Chris/TailDemography/TailDemography/Scripts and notes/Descriptive/'
                         ,'output':'S:/Chris/TailDemography/TailDemography/outputFiles/'},
              'silverSurfer':{'source':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles'
                              ,'log':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                              ,'output':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles\\Descriptive\\'}
              ,'dataPers':{'source':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
                           ,'log': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                           ,'output':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles/Descriptve/'}}

### Choose Device

In [3]:
device = deviceDict['silverSurfer']
device

{'source': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles',
 'log': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\',
 'output': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles\\Descriptive\\'}

# Source Data


### Logging

In [4]:
logging.basicConfig(filename=device['log']+'Desriptive Analyses.log'
                    , filemode='a',
                    format='%(funcName)s - %(levelname)s - %(message)s - %(asctime)s', level=logging.DEBUG)

## Functions

This section contains functions that were created for this notebook.

- [distribution](#distribution)
- [monthlit](#monthlit)

### distribution
[Back to Top](#TOC)

[Back to Functions](#Functions)

*distribution* takes a series or list of numeric objects, *x*, and returns descriptive stats of x including
        n, minimum, maximum, median, sIQR, mean, and stdev
    
Here are a few examples of how *distribution* works.

In [5]:
foo = [0,1,2,'r']
distribution(foo)

In [6]:
bar = [0,1,2]
distribution(bar)

,n,minimum,maximum,median,siqr,mean,stdev
0,3,0,2,1.0,0.5,1.0,1.0


[Back to Functions](#Functions)

## monthlit
[Back to Top](#TOC)

[Back to Functions](#Functions)

Here are a few examples of how _monthlit_ works.

In [7]:
dates = pd.DataFrame(data={'dates':['2018-12-9','2019-8-5', '2017/7/4',np.nan,None]})
dates.dates = pd.to_datetime(dates.dates)
dates

,dates
0,2018-12-09
1,2019-08-05
2,2017-07-04
3,NaT
4,NaT


In [8]:
np.isnan(np.nan)

True

In [9]:
monthlit(dates.dates.dt.month[0])

'Dec'

In [10]:
dates.dates.dt.month.apply(monthlit)

0    Dec
1    Aug
2    Jul
3    NaN
4    NaN
Name: dates, dtype: object

[Back to Functions](#Functions)

We'll display all files in the source folder with the prefix _'cleaned CC data 2000-2017'_. The file names will be saved in a variable, _mysourcefiles_.

## Get Data
[Top](#TOC)

Here we can set the locations from which we get data and to which we export it.

In [11]:
os.chdir(device['source'])
mysourcefiles = glob.glob('cleaned CC data 2000-2017_*.csv')
mysourcefiles

['cleaned CC data 2000-2017_2019-01-31 01hrs43min.csv',
 'cleaned CC data 2000-2017_2019-03-10 14hrs42min.csv',
 'cleaned CC data 2000-2017_2019-03-12 21hrs48min.csv',
 'cleaned CC data 2000-2017_2019-03-12 22hrs52min.csv',
 'cleaned CC data 2000-2017_2019-03-12 22hrs55min.csv',
 'cleaned CC data 2000-2017_2019-04-25 00hrs58min.csv',
 'cleaned CC data 2000-2017_2019-04-25 01hrs00min.csv',
 'cleaned CC data 2000-2017_2019-05-02 00hrs17min.csv',
 'cleaned CC data 2000-2017_2019-05-02 01hrs03min.csv',
 'cleaned CC data 2000-2017_2019-05-02 01hrs08min.csv',
 'cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv']

In [25]:
pd.to_datetime(mysourcefiles[0].split("_")[1].split(".csv")[0].split(' ')[0])

Timestamp('2019-01-31 00:00:00')

Automate getting the latest file

In [34]:
[latestFile for latestFile in mysourcefiles if \
 max({pd.to_datetime(afile.split("_")[1].split(".csv")[0].split(' ')[0]) \
      for afile in mysourcefiles}) == pd.to_datetime(latestFile.split("_")\
                                                     [1].split(".csv")[0].split(' ')[0])]

['cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv']

In [35]:
min({afile.split(' ')[-1].replace('hrs','').replace('min.csv','') for afile in mysourcefiles})

'0017'

In [36]:
latest = mysourcefiles[-1]
latest

'cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv'

The most recent file is the one we will use as _df_ in our descriptive analysis.

In [37]:
df=pd.read_csv(latest)
df.head()

,species,toes_orig,sex,date,svl,tl,rtl,autotomized,mass,location,meters,newRecap,painted,sighting,paint.mark,vial,misc,rtl_orig,toes,toe_pattern,description,action,pattern_b,replacement,year,tl_svl,mass_svl,all_meters,initialCaptureDate,year_diff,smallest_svl,svl_diff,liznumber,sex_count,daysSinceCapture,capture
0,j,3-7-11-19,m,2002-07-14,63.0,92.0,0.0,False,10.0,halfway up to site,-200,N,NaN,NaN,b101t,toes in vial 58-02,NaN,0.0,3-7-11-19,NaN,NaN,NaN,NaN,NaN,2002,1.460317,0.158730,['-200'],2002-07-14,0,63.0,0.0,375,1,0,1
1,j,3-7-11-18,m,2002-07-14,66.0,92.0,0.0,False,10.8,left downstream 100m v 1 falls,-100,N,NaN,NaN,b102t,toes in vial 59-02,NaN,0.0,3-7-11-18,NaN,NaN,NaN,NaN,NaN,2002,1.393939,0.163636,['-100' '87'],2002-07-14,0,66.0,0.0,374,1,0,1
2,j,3-7-12-16,m,2002-07-14,68.0,103.0,0.0,False,10.3,90m v 1 falls,-90,N,NaN,NaN,b103t,toes in vial 60-02,NaN,0.0,3-7-12-16,NaN,NaN,NaN,NaN,NaN,2002,1.514706,0.151471,['-90'],2002-07-14,0,68.0,0.0,376,1,0,1
3,j,10,m,2002-07-14,85.0,118.0,0.0,False,19.5,sb - trail intersection v 1 falls,-20,R,toe loss may be natural,NaN,b104t,NaN,NaN,0.0,10,NaN,NaN,NaN,NaN,NaN,2002,1.388235,0.229412,['-20' '-12' '-35'],2002-07-14,0,85.0,0.0,830,2,0,1
4,v,10-16,f,2002-07-03,63.0,49.0,21.0,True,11.6,left side @ base juniper 8 m ^ sb; 15m v 1 falls,-15,N,painted; gravid; <5 mites in pockets,NaN,w1a,toes in vial 34-02 (10-16),NaN,21.0,10-16,NaN,NaN,NaN,NaN,NaN,2002,0.777778,0.184127,['-15'],2002-07-03,0,63.0,0.0,1414,1,0,1


## Analyze Data
[Top](#TOC)

We will first examine the range and distribution of number of variables in our data set:
- [Population Size](#Population-Size)
- [Species Distribution](#Species-Distribution)
- [Sex Distribution](#Sex-Distribution)
- [Tail Condition Distribution](#Tail-Condition-Distribution)
- [Location](#Location)
- [Morphometrics](#Morphometrics):
    - [SVL](#SVL)
    - [TL](#TL)
    - [RTL](#RTL)
    - [Mass](#Mass)
- [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
- [Captures](#Captures)
- [Growth](#Growth)
    - [SVL Growth](#SVL-Growth)

### Population Size

We will begin by examining the range and distribution of _species_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [38]:
populationSize = df.groupby(['year','species']).liznumber.nunique().reset_index()
populationSize

,year,species,liznumber
0,2000,j,155
1,2001,j,137
2,2001,other,1
3,2002,j,121
4,2002,other,20
5,2002,v,80
6,2003,j,103
7,2003,other,25
8,2003,v,74
9,2004,j,73


In [39]:
Sjarrovii = go.Scatter(x = populationSize.loc[populationSize.species=='j'].year
           , y = populationSize.loc[populationSize.species=='j'].liznumber
          ,name = 'S. jarrovii')
Svirgatus = go.Scatter(x = populationSize.loc[populationSize.species=='v'].year
           , y = populationSize.loc[populationSize.species=='v'].liznumber
          ,name = 'S. virgatus')
data = [Sjarrovii, Svirgatus]
layout = go.Layout(
    title = 'Population Size for Sceloporus jarrovii and S. virgatus 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18))
)
fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Population Size for Sceloporus jarrovii and S. virgatus 2000-2017.html')

### Species Distribution

We will begin by examining the range and distribution of _species_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [40]:
df.species.value_counts(dropna=False)

j        1735
v         804
other     225
sj?         1
Name: species, dtype: int64

These values should only be _'j', 'v', and 'other'_ any other values will be removed in the cleaning notebook and excluded from further analyses here.

In [41]:
df = df.loc[df.species.isin(['j','v','other'])]
df.species.value_counts(dropna=False)

j        1735
v         804
other     225
Name: species, dtype: int64

### Sex Distribution

We will begin by examining the range and distribution of _sex_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [42]:
df.sex.value_counts(dropna=False)

f       1436
m       1297
NaN       26
n          2
male       1
f`         1
[m]        1
Name: sex, dtype: int64

These values should only be _'m' and 'f'_.  Any other values will be removed in the cleaning notebook and excluded from further analyses here.

In [43]:
df = df.loc[df.sex.isin(['m','f'])]
df.sex.value_counts(dropna=False)

f    1436
m    1297
Name: sex, dtype: int64

<a id = 'Autotomy'></a>

### Tail Condition Distribution

Here we look at the proportion of individuals in our data who have experienced autotomy.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [45]:
#need to get rid of duplicate lizard numbers in each year (in case of a change in autotomy status)
#set bin size to 1
jarrovii_aut = go.Bar(y = df.loc[(df.species=='j')&(df.autotomized==True)]\
                            .groupby(['year']).liznumber.nunique()
                     ,name = 'S. jarrovii (Autotomized)',x=df.year.unique())
jarrovii_int = go.Histogram(y = df.loc[(df.species=='j')&(df.autotomized==False)]\
                            .groupby(['year']).liznumber.nunique()
                     ,name = 'S. jarrovii (Intact)',x=df.year.unique())
virgatus_aut = go.Histogram(y = df.loc[(df.species=='v')&(df.autotomized==True)]\
                            .groupby(['year']).liznumber.nunique()
                     ,name = 'S. virgatus (Autotomized)',x=df.year.unique())
virgatus_int = go.Histogram(y = df.loc[(df.species=='v')&(df.autotomized==False)]\
                            .groupby(['year']).liznumber.nunique()
                     ,name = 'S. virgatus (Intact)',x=df.year.unique())
data = [jarrovii_aut, jarrovii_int, virgatus_aut, virgatus_int]
layout = go.Layout(
    title = 'Distribution of Autotomy for Sceloporus jarrovii and S. virgatus 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18))
)
fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Distribution of Autotomy for Sceloporus jarrovii and S. virgatus 2000-2017')

Let's look at how this distribution changes throughout the years.

<a id = 'resume'></a>

[Back to Top](#TOC)

### Morphometrics

In this section we describe the distributions of various morphometrics.

- [SVL](#SVL)
- [TL](#TL)
- [RTL](#RTL)
- [Mass](#SVL)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

#### SVL

Now we examine the range and distribution of svl values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of SVL](#SVLhist)

In [ ]:
print("svl values in the data set range from {} to {} for and are distributed across species and sex \
as displayed here:"\
      .format(df.svl.min(), df.svl.max()))
df.groupby(['species','sex']).svl.apply(distribution)

Let's plot these values. 

##### Histogram of SVL values

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)


In [46]:
femaleSj = go.Histogram(x = df.loc[(df.species=='j')&(df.sex=='f')].svl,name='Sj females')
maleSj = go.Histogram(x = df.loc[(df.species=='j')&(df.sex=='m')].svl,name='Sj males')
femaleSv = go.Histogram(x = df.loc[(df.species=='v')&(df.sex=='f')].svl,name='Sv females')
maleSv = go.Histogram(x = df.loc[(df.species=='v')&(df.sex=='m')].svl,name='Sv males')

data = [maleSj,maleSv,femaleSj,femaleSv]
layout = go.Layout(
    title = 'Histogram of SVL at Capture for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'SVL at time of capture',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of SVL at Capture for CC 2000-2017')

 Outliers will be addressed in the Cleaning notebook, but will be removed for the remained of the analyses here.

In [47]:
df.loc[(df.species=='v')&(df.svl==79)] = None
print("svl values in the data set range from {} to {} for and are distributed across species and sex \
as displayed here:"\
      .format(df.svl.min(), df.svl.max()))
df.groupby(['species','sex']).svl.apply(distribution)

svl values in the data set range from 13.0 to 112.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median  siqr       mean      stdev
species sex                                                             
j       f   0  941     13.0     87.0    65.0   8.5  61.760893  13.934161
        m   0  772     24.0     98.0    73.0  13.5  68.308290  17.837450
other   f   0  134     32.0    105.0    48.0   2.0  49.208955   9.789571
        m   0   88     36.0    112.0    49.0   1.5  51.045455  12.389195
v       f   0  361     31.0     66.0    56.0   3.5  54.930748   5.915908
        m   0  436     28.0     60.0    51.0   3.0  50.378440   4.821064

#### TL

Now we examine the range and distribution of TL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of TL](#TLhist)

In [48]:
print("tl values among intact lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df.tl.min(), df.tl.max()))
df.loc[df.autotomized==False].groupby(['species','sex']).tl.apply(distribution)

tl values among intact lizards in the data set range from 7.0 to 165.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median    siqr       mean      stdev
species sex                                                               
j       f   0  678     10.0    121.0    85.0  13.375  79.966077  19.370531
        m   0  529     12.0    132.0    96.0  22.000  88.584121  26.290859
other   f   0   92     37.0    129.0    67.0   4.125  67.576087  11.637599
        m   0   67     52.0    132.0    78.0   5.000  79.820896  14.075473
v       f   0  297     30.0     83.0    69.0   4.000  68.680135   7.630976
        m   0  343     20.0     86.0    69.0   5.000  67.110787   8.718433

In [49]:
print("tl values among autotomized lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df.tl.min(), df.tl.max()))
df.loc[df.autotomized==True].groupby(['species','sex']).tl.apply(distribution)

tl values among autotomized lizards in the data set range from 7.0 to 165.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median    siqr       mean      stdev
species sex                                                               
j       f   0  263      7.0    102.0    66.0  11.250  63.946768  19.118080
        m   0  243      7.0    125.0    76.0  13.250  72.786008  24.460870
other   f   0   42     13.0    165.0    54.0   5.500  57.523810  29.149939
        m   0   21     14.0     78.0    59.0  10.000  56.619048  14.941473
v       f   0   64     10.0     63.0    48.5   5.625  46.187500  10.852364
        m   0   92     12.0     68.0    49.0   7.125  46.021739  13.764086

#### RTL

Now we examine the range and distribution of RTL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of RTL](#RTLhist)

In [50]:
print("rtl values among autotomized lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df.rtl.min(), df.rtl.max()))
df.loc[df.autotomized==True].groupby(['species','sex']).rtl.apply(distribution)

rtl values among autotomized lizards in the data set range from 0.0 to 75.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median    siqr       mean      stdev
species sex                                                               
j       f   0  263      0.0     58.0    18.0  14.500  19.228137  15.786301
        m   0  243      0.0     60.0    21.0  15.750  20.473251  17.137096
other   f   0   42      0.0     75.0    17.5  14.125  21.119048  18.641944
        m   0   21      4.0     45.0    30.0   9.500  26.952381  12.874301
v       f   0   64      0.0     39.0    25.0  13.125  20.546875  13.266415
        m   0   92      0.0     40.0    17.5  10.625  18.097826  12.752125

## Captures

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.

In [51]:
df.groupby(['species','liznumber']).capture.max().reset_index().rename(columns={'capture':'numCaptures'})

,species,liznumber,numCaptures
0,j,1.0,2.0
1,j,2.0,4.0
2,j,3.0,3.0
3,j,4.0,8.0
4,j,5.0,3.0
5,j,6.0,2.0
6,j,7.0,2.0
7,j,8.0,5.0
8,j,9.0,3.0
9,j,10.0,3.0


In [52]:
Sj = go.Histogram(x = df.loc[(df.species=='j')].groupby('liznumber').capture.max(),name='Sj')
Sv = go.Histogram(x = df.loc[(df.species=='v')].groupby('liznumber').capture.max(),name='Sv')

data = [Sj,Sv]
layout = go.Layout(
    title = 'Histogram of Maximum Number of Captures for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Unique Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Number of Captures for CC 2000-2017')

## Reducing the analyses sample by date range and capture

In [53]:
# convert date to pandas datetime
#df.date=pd.to_datetime(df.date)
# limiting months to between May and August
# df = df.loc[(df.date.dt.month>=5) & (df.date.dt.month<=8)]
# limit to first captures
df_first = df.sort_values(by=['liznumber','date'])
df_first = df_first.loc[~df_first.duplicated(subset='liznumber')]

### Number of lizards (*Sj*) by year and sex

In [55]:
df.columns

Index(['species', 'toes_orig', 'sex', 'date', 'svl', 'tl', 'rtl',
       'autotomized', 'mass', 'location', 'meters', 'newRecap', 'painted',
       'sighting', 'paint.mark', 'vial', 'misc', 'rtl_orig', 'toes',
       'toe_pattern', 'description', 'action', 'pattern_b', 'replacement',
       'year', 'tl_svl', 'mass_svl', 'all_meters', 'initialCaptureDate',
       'year_diff', 'smallest_svl', 'svl_diff', 'liznumber', 'sex_count',
       'daysSinceCapture', 'capture'],
      dtype='object')

In [54]:
df.loc[df.].groupby('year').sex.value_counts()

year    sex
2000.0  f      116
        m       87
2001.0  f       94
        m       83
2002.0  f      150
        m      134
2003.0  f      152
        m      130
2004.0  f       91
        m       72
2005.0  m       98
        f       74
2006.0  m       87
        f       69
2007.0  f       89
        m       70
2008.0  f       74
        m       59
2009.0  f       87
        m       73
2010.0  m       45
        f       42
2011.0  f       36
        m       28
2012.0  f       50
        m       39
2013.0  f       58
        m       51
2014.0  f       58
        m       50
2015.0  f       84
        m       83
2016.0  m       60
        f       49
2017.0  f       63
        m       47
Name: sex, dtype: int64

Pull out all individuals that we've recaught for Sj and writes to csv

In [ ]:
multicapToes=df.loc[(df.species=='j')& (df.toes!="")& (df.toes!='NA')]\
.toes.value_counts()[df.loc[df.species=='j']\
                     .toes.value_counts()>1].index.tolist()
# df.loc[df.toes.isin(multicapToes)].sort_values(by=['toes','date']).to_csv('multicaps.csv')

### Maximum Number of Captures based on Tail condition
 - match for species, sex, size and location

In [56]:
#separate species
dfF = df.loc[df.sex =='f']
dfM = df.loc[df.sex =='m']

females = go.Histogram(x = dfF.groupby('liznumber').capture.max(),name='females')
males = go.Histogram(x = dfM.groupby('liznumber').capture.max(),name='males')

data = [males,females]
layout = go.Layout(
    title = 'Maximum Number of Captures per Individual 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Captures per Individual in Crystal Creek 2000 - 2017')

C:\Users\craga_eowcrpe\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [ ]:
df.capture.value_counts()

## Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.

### SVL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate?

In [ ]:
df.capture.value_counts()

## Export Files
[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)